In [2]:
import torch
import h5py
import sklearn
import numpy as np
import pandas as pd
from torch import nn
import sklearn.datasets
import sklearn.linear_model
import torch.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed

In [5]:
# Import small dataset (n=103)
data = pd.read_csv("slump_test.csv")

X = torch.FloatTensor(data.iloc[:, 1:10].values)
y = torch.FloatTensor(data.iloc[:, 10].values)
x_min = X.min(dim=0)[0]
x_max = X.max(dim=0)[0]
X = (X - x_min) / (x_max - x_min) # min max scaler

In [6]:
class ConcreteNN(nn.Module):

  def __init__(self, size):
    super().__init__()
    # layers
    self.input_layer = nn.Linear(size, size)     
    self.hidden_layer = nn.Linear(size, size)
    self.output_layer = nn.Linear(size, 1) 
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.input_layer(x)
    x = self.activation(x)
    x = self.hidden_layer(x)
    x = self.activation(x)
    x = self.output_layer(x)
    return x

def test_loss(model, X_test, y_test):
  model.eval()
  output = model(X_test)
  loss = sklearn.metrics.mean_squared_error(output.detach().numpy(), y_test.detach().numpy())
  return loss.item()

def pr_model(X_train, y_train):
    degree = 3
    polyreg = make_pipeline(PolynomialFeatures(degree),LinearRegression())
    inputs_reg = X_train[:, [0, 3, 5, 6]]
    polyreg.fit(inputs_reg, y_train)
    return torch.FloatTensor(polyreg.predict(inputs_reg))

def train_iml_cv(train_index, test_index):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = ConcreteNN(9)
    criterion = nn.MSELoss()
    iml_crit = nn.ReLU()
    optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum= 0.5)
    epochs = 2000
    lamda = 0.1
    loss_over_time = []
    test_loss_over_time = []

    # Training Loop
    for i in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train).flatten()
        loss = criterion(y_train, output) + lamda * iml_crit(torch.norm(pr_model(X_train, y_train)-output))
        loss_over_time.append(loss.item())
        test_loss_over_time.append(test_loss(model=model, X_test=X_test, y_test=y_test))
        loss.backward()
        optimizer.step()
    return test_loss_over_time[-1]

def train_iml_multirun(K, i):
    skf = sklearn.model_selection.KFold(n_splits=K)
    skf.get_n_splits(X, y)
    loss_test = []
    cv_loss_test = Parallel(n_jobs=-1)(delayed(train_iml_cv)(train_index, test_index) for train_index, test_index in skf.split(X, y))
    return cv_loss_test

In [ ]:
kfold = 10
multiruns = 50
loss = Parallel(n_jobs=-1)(delayed(train_iml_multirun)(multiruns, i) for i in range(kfold))

hf = h5py.File('polyreg_iml.h5', 'w')
hf.create_dataset('Loss IML', data=loss)

In [10]:
hf = h5py.File('polyreg_iml.h5', 'r')
loss = np.array(hf.get('Loss IML'))
loss_va = np.array(hf.get('Loss Vanilla'))
print(loss_va.mean())
print(loss.mean())

9.452442795142531
9.099921045333147
